## Settings

In [1]:
%matplotlib inline
import os
os.chdir('U:\\GitHubClones\\SALTSeniorDesign')
os.getcwd()
import Excel
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
import re
from datetime import datetime

In [3]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Path Setup For the File System

In [2]:
path = Excel.directory + "\\" + Excel.shelf + "\\" + Excel.deal + "\\" + Excel.dfTapes[Excel.deal][5]
print(path)

U:\AutoLoanTapes\Santander Drive Auto Receivables Trust\SDART 2017-1\2017\06 June\2017.07.17_Santander Drive Auto Receivables Trust 2017-1 Data Tape.csv


*Reading in CSV File*

In [3]:
df = pd.read_csv(path, memory_map=True, low_memory=False)
total = len(df.index)
print(total)
df['reportingPeriodActualEndBalanceAmount'].sum()

63447


1076979870.0699933

### Snap Shot of the Data

In [12]:
df.head().T

,0,1,2,3,4
assetTypeNumber,SC,SC,SC,SC,SC
assetNumber,"=""6345119""","=""634535""","=""6345360""","=""6345391""","=""634551"""
reportingPeriodBeginningDate,01-01-2017,01-01-2017,01-01-2017,01-01-2017,01-01-2017
reportingPeriodEndingDate,01-31-2017,01-31-2017,01-31-2017,01-31-2017,01-31-2017
originatorName,SC,SC,SC,SC,SC
originationDate,10/2013,03/2012,11/2013,11/2013,03/2012
originalLoanAmount,23260,12031,21332.3,24544.2,23589.1
originalLoanTerm,60,72,72,72,72
loanMaturityDate,01/2019,03/2018,11/2019,11/2019,09/2018
originalInterestRatePercentage,0.1194,0.18,0.181,0.0855,0.171


In [13]:
list(df.columns)

['assetTypeNumber',
 'assetNumber',
 'reportingPeriodBeginningDate',
 'reportingPeriodEndingDate',
 'originatorName',
 'originationDate',
 'originalLoanAmount',
 'originalLoanTerm',
 'loanMaturityDate',
 'originalInterestRatePercentage',
 'interestCalculationTypeCode',
 'originalInterestRateTypeCode',
 'originalInterestOnlyTermNumber',
 'originalFirstPaymentDate',
 'underwritingIndicator',
 'gracePeriodNumber',
 'paymentTypeCode',
 'subvented',
 'vehicleManufacturerName',
 'vehicleModelName',
 'vehicleNewUsedCode',
 'vehicleModelYear',
 'vehicleTypeCode',
 'vehicleValueAmount',
 'vehicleValueSourceCode',
 'obligorCreditScoreType',
 'obligorCreditScore',
 'obligorIncomeVerificationLevelCode',
 'obligorEmploymentVerificationCode',
 'coObligorIndicator',
 'paymentToIncomePercentage',
 'obligorGeographicLocation',
 'assetAddedIndicator',
 'remainingTermToMaturityNumber',
 'reportingPeriodModificationIndicator',
 'servicingAdvanceMethodCode',
 'reportingPeriodBeginningLoanBalanceAmount',
 '

### Feature Calculation Steps

In [47]:
WA = lambda x: np.average(x, axis=0, weights = df.loc[x.index, 'reportingPeriodActualEndBalanceAmount'])

In [48]:
df['LTV'] = (df['originalLoanAmount']/df['vehicleValueAmount'])*100 #Loan to value ratio
df['pctDeal'] = (df['reportingPeriodActualEndBalanceAmount']/(df['reportingPeriodActualEndBalanceAmount'].sum()))*100 #Individual Loan as a Percent of Current month Balance

print(df['pctDeal'].head())
df['LTV'].head()

0    0.000844
1    0.000158
2    0.001234
3    0.001038
4    0.000000
Name: pctDeal, dtype: float64


0    132.241913
1     88.626151
2    140.261095
3    148.303505
4    142.040463
Name: LTV, dtype: float64

array(['-', '4', '1', '3'], dtype=object)

## Selecting and Finding Data Characteristics

In [23]:
df['originalInterestRateTypeCode'].unique() #all of the loans are fixed rate loans
df['subvented'].unique() #1=rate, 2=cash, 0= none, 98=other
df['vehicleTypeCode'].unique() #1=car 2=truck 3=suv
df['vehicleNewUsedCode'].unique() #1=new 2=used
df['vehicleManufacturerName'].unique()
df['vehicleModelName'].unique()
df['vehicleValueAmount'].dtype #float
df['vehicleValueSourceCode'].unique() #98=other 3=KBB
df['obligorCreditScoreType'].unique() #only Bureau
df['obligorCreditScore'].dtype #float
df['obligorIncomeVerificationLevelCode'].unique() #3=Stated, verified but not to “level 4” or “level 5” 2=Stated, not verified
df['obligorEmploymentVerificationCode'].unique() # 1=not stated 2=stated not verified 3=verifited by 3rd party
df['paymentToIncomePercentage'].dtype #float
df['obligorGeographicLocation'].unique()
#df['assetAddedIndicator'].unique()
df['remainingTermToMaturityNumber'].dtype #int
df['reportingPeriodModificationIndicator'].unique()
df['servicingAdvanceMethodCode'].unique()
df['reportingPeriodBeginningLoanBalanceAmount'].dtype #float
df['totalActualAmountPaid'].dtype #float
df['actualInterestCollectedAmount'].dtype #float
df['actualPrincipalCollectedAmount'].dtype #float
df['zeroBalanceCode'].unique()
df['zeroBalanceEffectiveDate'].unique() #date at which the loan is considered dead
df['currentDelinquencyStatus'].unique() # How many days delinquent
df['assetSubjectDemandIndicator'].unique()
df['assetSubjectDemandStatusCode'].unique() #Asset has been repurchased replaced ect
df['repurchaseAmount'].unique()
df['repurchaseReplacementReasonCode'].unique() #reason why the car has thrown a zero balance code
df['chargedoffPrincipalAmount'].dtype #float
df['recoveredAmount'].dtype #float
df['modificationTypeCode'].unique() #what part of the loan has been modified
df['paymentExtendedNumber'].unique()
df['repossessedIndicator'].unique()
df['repossessedProceedsAmount'].dtype #object
df['LTV'].dtype #float
df['pctDeal'].dtype #float

dtype('float64')

### Data Cleaning Steps

In [4]:
noScore = df[df.obligorCreditScore.isin(['-'])]
noScore['obligorCreditScore'].describe()

count     9073
unique       1
top          -
freq      9073
Name: obligorCreditScore, dtype: object

In [50]:
print(df['obligorCreditScore'].describe())
df['obligorCreditScore'] = pd.to_numeric(df['obligorCreditScore'], errors='coerce')
print(df['obligorCreditScore'].describe())

count     60291
unique      483
top           -
freq       8513
Name: obligorCreditScore, dtype: object
count    51778.000000
mean       602.909189
std         64.481956
min        385.000000
25%        562.000000
50%        600.000000
75%        637.000000
max        900.000000
Name: obligorCreditScore, dtype: float64


### Bin Selection for Strat Tables

In [51]:
FicoBins = [-1, 440, 500, 560, 620, 680, 740, 800, 1000]
APRBins = [0, .02 ,.04, .06, .08, .1, .12, .14, .16, 1000]
APRBins4 = [0, .04, .08, .12, .16, .20, .24, .28, 1000]
LTVBins = [-1, .7, .8, .9, 1, 1.1, 1.2, 1.3, 1.4, 1000]
PTIBins = [-1, .02, .04, .06, .08, .1, .12, .14, .16, 1000]

In [52]:
FicoGroup = ['[-1, 440)', '[440, 500)', '[500, 560)', '[560, 620)', '[620, 680)', '[680, 740)', '[740, 800)', '[800, 1000)']
APRGroup = ['[0.0, 0.02)', '[0.02, 0.04)', '[0.04, 0.06)', '[0.06, 0.08)', '[0.08, 0.1)', '[0.1, 0.12)', '[0.12, 0.14)', '[0.14, 0.16)', '[0.16, 1000.0)']
APRGroup4 = ['[0.0, 0.04)', '[0.04, 0.08)', '[0.08, 0.12)', '[0.12, 0.16)', '[0.16, 0.20)', '[0.20, 0.24)', '[0.24, 0.28)', '[0.28, 1000)']
PTIGroup = ['[-1.0, 0.02)', '[0.02, 0.04)', '[0.04, 0.06)', '[0.06, 0.08)', '[0.08, 0.1)', '[0.1, 0.12)', '[0.12, 0.14)', '[0.14, 0.16)', '[0.16, 1000.0)']

LTVGroup = ['[-1.0, 0.7)', '[0.7, 0.8)', '[0.8, 0.9)', '[0.9, 1.0)', '[1.0, 1.1)', '[1.1, 1.2)', '[1.2, 1.3)', '[1.3, 1.4)', '[1.4, 1000.0)']

### Cutting the Data Based on Bin Size

In [53]:
df['Fico Group'] = pd.cut(df['obligorCreditScore'], FicoBins, right=True, labels=FicoGroup)
df['APR Group'] = pd.cut(df['originalInterestRatePercentage'], APRBins, labels=APRGroup)
df['APR Group4'] = pd.cut(df['originalInterestRatePercentage'], APRBins4, labels=APRGroup4)
df['PTI Group'] = pd.cut(df['paymentToIncomePercentage'], PTIBins, labels=PTIGroup)
df['Fico Group'].head()

0    [620, 680)
1           NaN
2    [500, 560)
3    [680, 740)
4    [620, 680)
Name: Fico Group, dtype: category
Categories (8, object): [[-1, 440) < [440, 500) < [500, 560) < [560, 620) < [620, 680) < [680, 740) < [740, 800) < [800, 1000)]

#### Added No Score Category

In [54]:
df['Fico Group'] = df['Fico Group'].cat.add_categories('No Score').fillna('No Score')

In [55]:
df['Fico Group'].head()

0    [620, 680)
1      No Score
2    [500, 560)
3    [680, 740)
4    [620, 680)
Name: Fico Group, dtype: category
Categories (9, object): [[-1, 440) < [440, 500) < [500, 560) < [560, 620) ... [680, 740) < [740, 800) < [800, 1000) < No Score]

### Ordered Dictionary for Loan Characteristics in Seclected Bins

In [56]:
a = OrderedDict([
    ('assetNumber',['count']),
    ('reportingPeriodActualEndBalanceAmount', ['sum','mean']),
    ('pctDeal', ['sum']),
    ('originalLoanTerm',[WA]),
    ('originalInterestRatePercentage',[WA])
])

In [57]:
df[['assetNumber', 'pctDeal','reportingPeriodActualEndBalanceAmount','originalLoanTerm','originalInterestRatePercentage']].groupby([df['Fico Group'], df['APR Group4']]).agg(a)

assetNumber reportingPeriodActualEndBalanceAmount                  pctDeal originalLoanTerm originalInterestRatePercentage
                               count                                   sum          mean        sum         <lambda>                       <lambda>
Fico Group  APR Group4                                                                                                                             
[-1, 440)   [0.08, 0.12)           2                          3.707164e+04  18535.820000   0.003808        74.113460                       0.117443
            [0.12, 0.16)           9                          1.168860e+05  12987.332222   0.012006        70.663388                       0.143945
            [0.16, 0.20)          40                          5.216711e+05  13041.777250   0.053582        71.150105                       0.181397
            [0.20, 0.24)          48                          5.230345e+05  10896.552292   0.053722        70.956498                       0.217354
            [0.24, 0.28)          27                          2.796532e+05  10357.525556   0.028724        68.795704                       0.252990
[440, 500)  [0.0, 0.04)            1                          1.127083e+04  11270.830000   0.001158        72.000000                       0.039000
            [0.04, 0.08)          14                          2.154939e+05  15392.423571   0.022134        72.619339                       0.068430
            [0.08, 0.12)          61                          1.019707e+06  16716.515902   0.104736        72.079275                       0.103020
            [0.12, 0.16)         205                          3.298666e+06  16091.052488   0.338811        71.569338                       0.145090
            [0.16, 0.20)         757                          9.883595e+06  13056.267952   1.015159        71.232997                       0.180749
            [0.20, 0.24)         640                          7.294430e+06  11397.547547   0.749222        70.740058                       0.217108
            [0.24, 0.28)         343                          3.413102e+06   9950.735802   0.350565        70.250083                       0.255185
            [0.28, 1000)           1                          1.479680e+03   1479.680000   0.000152        60.000000                       0.291700
[500, 560)  [0.0, 0.04)            9                          1.118481e+05  12427.570000   0.011488        71.128167                       0.033289
            [0.04, 0.08)         175                          3.308494e+06  18905.677943   0.339820        71.628673                       0.069268
            [0.08, 0.12)         838                          1.695862e+07  20237.016551   1.741846        71.684470                       0.103839
            [0.12, 0.16)        1992                          3.588122e+07  18012.662610   3.685416        71.889690                       0.144105
            [0.16, 0.20)        4405                          6.625491e+07  15040.841750   6.805143        71.324167                       0.178814
            [0.20, 0.24)        2278                          2.733928e+07  12001.439662   2.808059        70.264688                       0.216580
            [0.24, 0.28)         696                          7.209812e+06  10358.925302   0.740531        69.976394                       0.254603
            [0.28, 1000)           1                          1.110304e+04  11103.040000   0.001140        72.000000                       0.290200
[560, 620)  [0.0, 0.04)           99                          2.107045e+06  21283.281414   0.216418        71.363852                       0.031867
            [0.04, 0.08)         760                          1.737881e+07  22866.855079   1.785004        72.494772                       0.067985
            [0.08, 0.12)        2921                          6.133934e+07  20999.430924   6.300258        72.159379                       0.102630
            [0.12, 0.16)     

In [58]:
df[['assetNumber', 'pctDeal','reportingPeriodActualEndBalanceAmount','originalLoanTerm', 'originalInterestRatePercentage']].groupby([df['Fico Group']]).agg(a)

assetNumber reportingPeriodActualEndBalanceAmount                  pctDeal originalLoanTerm originalInterestRatePercentage
                  count                                   sum          mean        sum         <lambda>                       <lambda>
Fico Group                                                                                                                            
[-1, 440)           126                          1.478316e+06  11732.670000   0.151840        70.672053                       0.203097
[440, 500)         2022                          2.513775e+07  12432.119347   2.581936        71.046533                       0.192554
[500, 560)        10394                          1.570753e+08  15112.111610  16.133444        71.252306                       0.170440
[560, 620)        21560                          3.621409e+08  16796.889387  37.196052        71.455653                       0.152681
[620, 680)        12771                          2.360807e+08  18485.688926  24.248215        71.521659                       0.145215
[680, 740)         3268                          6.050452e+07  18514.235236   6.214512        71.010331                       0.132310
[740, 800)          921                          1.754118e+07  19045.802975   1.801682        70.646446                       0.103721
[800, 1000)         716                          1.372809e+07  19173.309874   1.410033        70.050277                       0.072938
No Score           8513                          9.991366e+07  11736.598244  10.262286        70.076109                       0.202186

### Formatting Rules

In [ ]:
("%.2f%%" % df['LTV'])